In [61]:
import pandas as pd
import numpy as np

import requests
from geopy.geocoders import Nominatim
from astral import LocationInfo
import datetime
from astral.sun import sun
from astral import LocationInfo

from time import time
import warnings

In [5]:
cd

/Users/karatechop


In [6]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/Formatted with YOE


In [7]:
ls

ALL-BPD-AB953.csv         ALL-MARIN-AB953.csv       ALL-SDPD-AB953.csv
ALL-DAVIS-AB953.csv       ALL-OAKLAND_PD-AB953.csv  ALL-SFPD-AB953.csv


In [170]:
pd.to_datetime(SDPD_later['time'], format='%H:%M:%S', exact=False)

0        1900-01-01 19:31:37
12       1900-01-01 16:56:56
19       1900-01-01 19:58:29
27       1900-01-01 20:04:49
44       1900-01-01 15:10:33
                 ...        
246643   1900-01-01 23:50:00
246644   1900-01-01 15:10:00
246646   1900-01-01 15:25:00
246649   1900-01-01 15:40:00
246651   1900-01-01 16:00:00
Name: time, Length: 104512, dtype: datetime64[ns]

## FUNCTION SETUP

In [269]:
def add_dusk_cols(df,date_col,time_col,city,city_lat,city_lon):
    cols = df.columns
    
    try:
        df[date_col] = pd.to_datetime(df[date_col])
    except:
        pass
    
    try:
        df[time_col] = pd.to_datetime(df[time_col], format='%H:%M:%S', exact=False)
    except: 
        print('time failed')
        pass
    
#     print('debug',df.dtypes)
    
    
    date_to_subtract = datetime.datetime(1900, 1, 1)
    
    dawn_list = []
    sunrise_list = []
    sunset_list = []
    dusk_list = []

    loc = LocationInfo(name=city, region='CA, USA', timezone='America/Los_Angeles', #https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
                       latitude=city_lat, longitude=city_lon)

    print('debug',loc)
    
    for i in df[date_col]:
        year = i.year
        month = i.month
        day = i.day


        s = sun(loc.observer, date=datetime.date(year, month, day), tzinfo=loc.timezone)
        sunset_list.append(s['sunset'].time())
        dusk_list.append(s['dusk'].time())


    sunset_list2 = [str(i)[:8] for i in sunset_list]  
    dusk_list2 = [str(i)[:8] for i in dusk_list]  


    df['sunsettime'] = sunset_list2
    df['dusktime'] = dusk_list2

    
    

    df['mins_sunset'] = [round(((datetime.datetime.strptime(i, '%H:%M:%S') - date_to_subtract).total_seconds() / 60.0),0) 
                         for i in df['sunsettime']]
    df['mins_dusk'] = [round(((datetime.datetime.strptime(i, '%H:%M:%S') - date_to_subtract).total_seconds() / 60.0),0) 
                       for i in dusk_list2]

    df['mins_stop_time'] = [round(((i - date_to_subtract).total_seconds()/60),0) 
                                for i in df[time_col]]
      
    df['mins_before_dusk'] = df['mins_dusk'] - df['mins_stop_time']
    
    
    
    
    df['itp'] = 0
    df.loc[(df['mins_stop_time']>=df['mins_dusk'].min()) &\
           (df['mins_stop_time']<=df['mins_dusk'].max()), 'itp'] = 1
    
    df['dark'] = 0
    df.loc[df['mins_before_dusk']<=0,'dark']=1

## WAVE 1

### SFPD

In [267]:
SFPD = pd.read_csv('ALL-SFPD-AB953.csv')
SFPD['date'] = pd.to_datetime(SFPD['date'])
SFPD_later = SFPD[SFPD['time']>'15:00:00']
SFPD_later.columns

Index(['date', 'time', 'Experience', 'race', 'type', 'location'], dtype='object')

In [268]:
warnings.filterwarnings("ignore") 
add_dusk_cols(SFPD_later,'date','time','San Francisco',37.7749,-122.4194)

debug date          datetime64[ns]
time          datetime64[ns]
Experience           float64
race                  object
type                  object
location              object
dtype: object
debug LocationInfo(name='San Francisco', region='CA, USA', timezone='America/Los_Angeles', latitude=37.7749, longitude=-122.4194)


In [271]:
SFPD_later.to_csv('ALL-SFPD-AB953-DUSK.csv')
SFPD_later.head(2)

,date,time,Experience,race,type,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark
0,2018-07-01,1900-01-01 18:11:00,3.0,hispanic/latino(a),1 Moving,MISSION ST & 8TH ST,20:35:16,21:07:00,1235.0,1267.0,1091.0,176.0,1,0
2,2018-07-01,1900-01-01 16:26:00,1.0,hispanic/latino(a),2 Equipment,SUNBEAM LN & JUNIOR TER,20:35:16,21:07:00,1235.0,1267.0,986.0,281.0,0,0


### SDPD

In [272]:
SDPD = pd.read_csv('ALL-SDPD-AB953.csv')
SDPD_later = SDPD[SDPD['time']>'15:00:00']
SDPD_later.columns

Index(['date', 'time', 'Experience', 'race', 'type', 'location'], dtype='object')

In [273]:
warnings.filterwarnings("ignore") 
add_dusk_cols(SDPD_later,'date','time','San Diego',32.7157,-117.1611)

debug LocationInfo(name='San Diego', region='CA, USA', timezone='America/Los_Angeles', latitude=32.7157, longitude=-117.1611)


In [274]:
SDPD_later.to_csv('ALL-SDPD-AB953-DUSK.csv')
print(SDPD_later.shape)
SDPD_later.head(2)

(104512, 14)


,date,time,Experience,race,type,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark
0,2018-07-15,1900-01-01 19:31:37,26,white,1 Moving,313.0,19:57:07,20:25:32,1197.0,1226.0,1172.0,54.0,1,0
12,2019-02-02,1900-01-01 16:56:56,28,hispanic/latino/a,1 Moving,712.0,17:22:03,17:48:21,1042.0,1068.0,1017.0,51.0,0,0


## WAVE 2

In [276]:
ls

ALL-BPD-AB953-DUSK.csv      ALL-OAKLAND-AB953-DUSK.csv
ALL-BPD-AB953.csv           ALL-OAKLAND_PD-AB953.csv
ALL-DAVIS-AB953-DUSK.csv    ALL-SDPD-AB953-DUSK.csv
ALL-DAVIS-AB953.csv         ALL-SDPD-AB953.csv
ALL-MARIN-AB953-DUSK.csv    ALL-SFPD-AB953-DUSK.csv
ALL-MARIN-AB953.csv         ALL-SFPD-AB953.csv


### OAKLAND

In [277]:
OAK = pd.read_csv('ALL-OAKLAND_PD-AB953.csv')
OAK_later = OAK[OAK['time']>'15:00:00']
OAK_later.columns

Index(['date', 'time', 'race', 'race perceived prior', 'type', 'Experience',
       'location'],
      dtype='object')

In [278]:
warnings.filterwarnings("ignore") 
add_dusk_cols(OAK_later,'date','time','Oakland',37.8044,-122.2712)

debug LocationInfo(name='Oakland', region='CA, USA', timezone='America/Los_Angeles', latitude=37.8044, longitude=-122.2712)


In [279]:
OAK_later.to_csv('ALL-OAKLAND-AB953-DUSK.csv')
OAK_later.head(2)

,date,time,race,race perceived prior,type,Experience,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark
0,2022-06-18,1900-01-01 22:16:00,Black/African American,No,Moving violation,NaN,20X,20:33:33,21:05:31,1234.0,1266.0,1336.0,-70.0,0,1
1,2022-04-20,1900-01-01 16:10:00,Black/African American,Yes,Moving violation,NaN,32X,19:49:22,20:17:24,1189.0,1217.0,970.0,247.0,0,0


### BERKELEY

In [189]:
OAK = pd.read_csv('ALL-OAKLAND_PD-AB953.csv')
OAK_later = OAK[OAK['time']>'15:00:00']
OAK_later.columns

Index(['date', 'time', 'race', 'race perceived prior', 'type', 'Experience',
       'location'],
      dtype='object')

In [190]:
warnings.filterwarnings("ignore") 
add_dusk_cols(OAK_later,'date','time','Oakland',37.8044,-122.2712)

debug date                    datetime64[ns]
time                    datetime64[ns]
race                            object
race perceived prior            object
type                            object
Experience                     float64
location                        object
dtype: object
debug LocationInfo(name='Oakland', region='CA, USA', timezone='America/Los_Angeles', latitude=37.8044, longitude=-122.2712)


In [192]:
OAK_later.to_csv('ALL-OAKLAND-AB953-DUSK.csv')
OAK_later.head(2)

,date,time,race,race perceived prior,type,Experience,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,dark
0,2022-06-18,1900-01-01 22:16:00,Black/African American,No,Moving violation,NaN,20X,20:33:33,21:05:31,1234.0,1266.0,1336.0,-70.0,1
1,2022-04-20,1900-01-01 16:10:00,Black/African American,Yes,Moving violation,NaN,32X,19:49:22,20:17:24,1189.0,1217.0,970.0,247.0,0


## WAVES 3-4

### BERKELEY

In [280]:
berk = pd.read_csv('ALL-BPD-AB953.csv')
berk['time'] = berk['time'].astype(str) + ':00'
berk_later = berk[berk['time']>'15:00:00']
berk_later.columns

Index(['date', 'time', 'race', 'race perceived prior', 'type', 'location'], dtype='object')

In [281]:
warnings.filterwarnings("ignore") 
add_dusk_cols(berk_later,'date','time','Berkeley',37.8715,-122.2730)

debug LocationInfo(name='Berkeley', region='CA, USA', timezone='America/Los_Angeles', latitude=37.8715, longitude=-122.273)


In [282]:
berk_later.to_csv('ALL-BPD-AB953-DUSK.csv')
berk_later.head(2)

,date,time,race,race perceived prior,type,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark
3,2020-10-01,1900-01-01 22:26:00,hispanic/latino,False,Non-moving violation / including registration,"37.87743879, -122.2784665",18:50:54,19:17:35,1131.0,1158.0,1346.0,-188.0,0,1
4,2020-10-01,1900-01-01 23:18:00,black/african american,False,Moving Violation,"37.85650341, -122.2948728",18:50:54,19:17:35,1131.0,1158.0,1398.0,-240.0,0,1


### MARIN

In [283]:
marin = pd.read_csv('ALL-MARIN-AB953.csv')
marin_later = marin[marin['time']>'15:00:00']
marin_later.columns

Index(['date', 'time', 'race', 'type', 'race perceived prior', 'experience',
       'location'],
      dtype='object')

In [284]:
warnings.filterwarnings("ignore") 
add_dusk_cols(marin_later,'date','time','Marin',38.0834,-122.7633)

debug LocationInfo(name='Marin', region='CA, USA', timezone='America/Los_Angeles', latitude=38.0834, longitude=-122.7633)


In [285]:
marin_later.to_csv('ALL-MARIN-AB953-DUSK.csv')
marin_later.head(2)

,date,time,race,type,race perceived prior,experience,location,sunsettime,dusktime,mins_sunset,mins_dusk,mins_stop_time,mins_before_dusk,itp,dark
0,2020-09-30 22:30:00,1900-01-01 22:30:00,hispanic/latino(a),2_Equipment violation,NaN,0.0,2000 sir Francis drake Blvd,18:54:19,19:21:04,1134.0,1161.0,1350.0,-189.0,0,1
5,2020-10-01 15:50:00,1900-01-01 15:50:00,hispanic/latino(a),1_Moving violation,NaN,17.0,3000 bridge way,18:52:47,19:19:32,1133.0,1160.0,950.0,210.0,0,0
